# CS 224N Final Project - Evaluating on CBT Dataset
By: Joseph O'Brien, Christopher Pondoc, Joseph Guman

In [1]:
import torch
print("Using GPU: " + str(torch.cuda.is_available()))

Using GPU: True


## Load in GPT-2 Model to Evaluate + R2G and G2G
All imported via models we've trained and HuggingFace Transformers.

In [2]:
from transformers import AutoTokenizer, GPT2LMHeadModel, RobertaModel, GPT2Tokenizer, RobertaTokenizer

# Model we will be editing
tokenizer = GPT2Tokenizer.from_pretrained("/home/ubuntu/test/weights/G2GMaskingT")
model = GPT2LMHeadModel.from_pretrained("/home/ubuntu/test/weights/G2GMaskingBestM").to("cuda")

## Load in 1:1 Real to Fake Words
Load in from the `.csv` file.

In [3]:
import pandas as pd

# Import CSV and get the real to fake
rtf_df = pd.read_csv("/home/ubuntu/test/datasets/realtofake.csv")
final_real_words = rtf_df["Real"].tolist()
final_fake_words = rtf_df["Fake"].tolist()

# Populate dictionary
real_to_fake_dict = {}
for i in range(len(final_real_words)):
    real_to_fake_dict[final_real_words[i]] = final_fake_words[i]

## Load in Children's Book Test Dataset
Also taken from HuggingFace

In [4]:
import pandas
cbt_df = pd.read_csv("/home/ubuntu/test/cbt/cbt_extract/cbt_csv_files/updated_cbt_info.csv")

## Evaluating on One Example
Writing a function that is reusable and works for one example

In [5]:
def evaluate_cbt(example):
    # extract the sentence
    sentence = example["sentences"]
    sentence = sentence.replace('"','')
    sentence = sentence.replace("\n","")
    sentence = sentence[1:len(sentence)-1]

    # extract the question
    question = example["question"]

    # make the total query with the sentence and the question
    total_query = sentence + " " + question

    
    #replace words in example with winodict fake words (currently, we replace all possible identified examples)
    replace_words = example["winodict_words"]
    replace_words = replace_words[1:len(replace_words)-1].replace("'","").split(",")

    for word in replace_words:
        word = word.strip()
        word = " "+word+" "
        while word in total_query:
            total_query = total_query.replace(word," "+real_to_fake_dict[word.strip()]+" ")
    

    # get the answer options for the model into a list
    options = example["options"].replace("'","")
    options = options.replace("\n","")
    options = options[1:len(options)-1].split(" ")

    # get the answer to the problem
    answer = example["answer"]

    
    if answer in replace_words:
        answer = real_to_fake_dict[answer][0]
    
    

    # initialize best answer and best loss (will be lowest value)
    best_answer = ""
    best_loss = float("inf")

    # compute the loss for each option (using full scoring)
    for option in options:
        
        if option in replace_words:
            option = real_to_fake_dict[option][0]
        

        updated_query = total_query.replace("XXXXX", option)
        
        # Tokenize each string and produce labels
        updated_input = tokenizer(updated_query, return_tensors="pt").to("cuda")

        """
        if len(updated_input[0]) > 1024:
            return -1
        """

        current_loss = model(**updated_input, labels=updated_input["input_ids"].to("cuda")).loss
        
        if current_loss < best_loss:
            best_answer = option
            best_loss = current_loss
    return best_answer == answer 



## Evaluating CBT on GPT-2

In [6]:
eval_indices = []

with open("/home/ubuntu/test/cbt/cbt_eval_info/eval_indices", "r") as f:
    eval_indices = f.read()

eval_indices = eval_indices[1:len(eval_indices)-1].split(", ")
for i in range(len(eval_indices)):
    eval_indices[i] = int(eval_indices[i])

In [7]:
# evaluating gpt-2 AFTER REPLACING words and saving indices and stats
correct_list = []
total_list = []

correct = 0 
total = 0 
for index, row in cbt_df.iterrows():
    #print(index)

    if index not in eval_indices:
        continue

    update = evaluate_cbt(row)
    if update == -1:
        continue
    if update == 1:
        correct_list.append(index)
    total_list.append(index)
    correct += update
    total += 1
    print(total)
    if total == 200:
        break

correct_list = str(correct_list)
total_list = str(total_list)

"""
with open("/home/ubuntu/test/CBT/cbt_eval_info/GPT2-10EpochsFinetined/GPT2-10EpochsFinetuned_correct", "w") as file1:
    file1.write(correct_list)

with open("/home/ubuntu/test/CBT/cbt_eval_info/GPT2-10EpochsFinetined/GPT2-10EpochsFinetuned_total", "w") as file2:
    file2.write(total_list)
"""


print("")
print("Number correct: ", correct)
print("Total: ", total)
print("Correct percentage: ",correct/total)

information_list = str([correct, total, correct/total])

"""
with open("/home/ubuntu/test/CBT/cbt_eval_info/GPT2-10EpochsFinetined/GPT2-10EpochsFinetuned_statistics", "w") as file3:
    file3.write(information_list)
"""


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200

Number correct:  17
Total:  200
Correct percentage:  0.085


'\nwith open("/home/ubuntu/test/CBT/cbt_eval_info/GPT2-10EpochsFinetined/GPT2-10EpochsFinetuned_statistics", "w") as file3:\n    file3.write(information_list)\n'